### Download model and tokneizer

In [3]:
from modelscope import snapshot_download, AutoTokenizer
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
import torch

# 在modelscope上下载Qwen模型到本地目录下
model_dir = snapshot_download("qwen/Qwen2-1.5B-Instruct", cache_dir="./", revision="master")

# Transformers加载模型权重
tokenizer = AutoTokenizer.from_pretrained("./qwen/Qwen2-1___5B-Instruct/", use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("./qwen/Qwen2-1___5B-Instruct/", device_map="auto", torch_dtype=torch.bfloat16)
model.enable_input_require_grads()  # 开启梯度检查点时，要执行该方法

2025-05-21 21:28:08,167 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


2025-05-21 21:28:08,385 - modelscope - INFO - Target directory already exists, skipping creation.


### Install ipywidgets

In [2]:
pip install ipywidgets

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 4.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]3 [ipywidgets]
Note: you may need to restart the kernel to use updated packages.


###  Preprocess training data

In [4]:
def process_func(example):
    """
    将数据集进行预处理
    """
    MAX_LENGTH = 384
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(
        f"<|im_start|>system\n你是一个文本分类领域的专家，你会接收到一段文本和几个潜在的分类选项，请输出文本内容的正确类型<|im_end|>\n<|im_start|>user\n{example['input']}<|im_end|>\n<|im_start|>assistant\n",
        add_special_tokens=False,
    )
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = (
        instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    )
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = (
        [-100] * len(instruction["input_ids"])
        + response["input_ids"]
        + [tokenizer.pad_token_id]
    )
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

### Load dataset 

In [7]:
import json
import pandas as pd
import os

from datasets import Dataset

# Read 1000 items training 
train_jsonl_new_path = "./datasets/new_train.jsonl"
test_jsonl_new_path = "./datasets/new_test.jsonl"
train_df = pd.read_json(train_jsonl_new_path, lines=True)[:1000]  # 取前1000条做训练（可选）
test_df = pd.read_json(test_jsonl_new_path, lines=True)[:10]  # 取前10条做主观评测

train_ds = Dataset.from_pandas(train_df)
train_dataset = train_ds.map(process_func, remove_columns=train_ds.column_names)

Map: 100%|█████████████████████████████████████████████████████████████| 1000/1000 [02:22<00:00,  7.02 examples/s]


### Using LORA method for traning

In [8]:
from peft import LoraConfig, TaskType, get_peft_model
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    inference_mode = False, # Traning mode
    r = 8, # LoRA 的秩（rank）
    lora_alpha=32, # Lora alaph,
    lora_dropout=0.1, # Dropoout raido
)

model = get_peft_model(model, config)


### Traning

In [10]:
args = TrainingArguments(
    output_dir = "./output/Qwen2",
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,
    logging_steps  = 10,
    num_train_epochs = 2,
    save_steps = 100,
    learning_rate = 1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="none",
)

### Config swanLab

In [12]:
import swanlab
from swanlab.integration.transformers import SwanLabCallback

# 实例化SwanLabCallback
swanlab_callback = SwanLabCallback(
    project="Qwen2-finetune", 
    experiment_name="Qwen2-1.5B-Instruc",
    description="Qwen2-1.5B-Instruct Traning",
        config={
        "model": "qwen/Qwen2-1.5B-Instruct",
        "dataset": "huangjintao/zh_cls_fudan-news",
    },
)


### Start Traning

In [13]:
trainer = Trainer(
    model = model,
    args=args,
    train_dataset=train_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[swanlab_callback],
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


swanlab: Using SwanLab to track your experiments. Please refer to https://docs.swanlab.cn for more information.
swanlab: (1) Create a SwanLab account.
swanlab: (2) Use an existing SwanLab account.
swanlab: (3) Don't visualize my results.


swanlab: Enter your choice:  2


swanlab: You chose 'Use an existing swanlab account'
swanlab: Logging into https://swanlab.cn
swanlab: You can find your API key at: https://swanlab.cn/space/~/settings
swanlab: Paste an API key from your profile and hit enter, or press 'CTRL + C' to quit: 


 ········


swanlab: Tracking run with swanlab version 0.5.8                                   
swanlab: Run data will be saved locally in /home/gongbiao/code/learning/finetune/swanlog/run-20250521_220659-a3b1799d
swanlab: 👋 Hi easy, welcome to swanlab!
swanlab: Syncing run Qwen2-1.5B-Instruc to the cloud
swanlab: 🏠 View project at https://swanlab.cn/@easy/Qwen2-finetune
swanlab: 🚀 View run at https://swanlab.cn/@easy/Qwen2-finetune/runs/blun8oouxlrhpxrsdvr7n


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,2.768000
20,0.413200
30,0.289300
40,0.265900
50,0.118800
60,0.031500
70,0.260100
80,0.008200
90,0.004700
100,0.073700


TrainOutput(global_step=126, training_loss=0.33604054073507467, metrics={'train_runtime': 1931.0669, 'train_samples_per_second': 1.036, 'train_steps_per_second': 0.065, 'total_flos': 6080592347136000.0, 'train_loss': 0.33604054073507467, 'epoch': 2.0})

### Predict validation

In [17]:
def predict(messages, model, tokenizer):
    device = "cuda"
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512)
    generated_ids = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(response)

    return response
    


In [18]:
test_text_list = []
for index, row in test_df.iterrows():
    instruction = row["instruction"]
    input_value = row["input"]

    messages = [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{input_value}"},
    ]

    response = predict(messages, model, tokenizer)
    messages.append({"role": "assistant", "content": f"{response}"})
    result_text = f"{messages[0]}\n\n{messages[1]}\n\n{messages[2]}"
    test_text_list.append(swanlab.Text(result_text, caption=response))

swanlab.log({"Prediction": test_text_list})
swanlab.finish()

Art
Military
Politics
外国文学研究
History
Space
Transport
Literature
Economy
Art
swanlab: 🏠 View project at https://swanlab.cn/@easy/Qwen2-finetune
swanlab: 🚀 View run at https://swanlab.cn/@easy/Qwen2-finetune/runs/blun8oouxlrhpxrsdvr7n
                                                                                                    